### Use Level Set method to extract distance function from the plasma boundary

In [17]:
from firedrake import *
from firedrake.pyplot import *
import matplotlib.pyplot as plt

In [18]:
def G(R, psi_norm):
    r0 = 6.2
    alpha = 2.0
    beta = 0.5978
    gamma = 1.395
    lambda_ = 1.365461e6
    return lambda_ * (beta * R / r0 + (1 - beta) * r0 / R) * abs(1 - psi_norm**alpha) ** gamma

params = {
    "geometry": "ITER",
    "I": [-8.208e5, -8.621e4, 2.783e6, 1.703e6, -6.491e5, -7.971e5, -2.026e6, -2.508e6, -2.15e6, -1.874e6, -3.607e6, -2.303e6],
    "j_cv": 0,                # Vessel wall current density
    "function_space_family": "P",
    "function_space_degree": 1,
    "max_iterations": 1000,
    "tolerance": 1e-5,
    "verbose": True,
    "G": G,
    # Initial guess (can be a Constant or a Firedrake Function)
    "initial_guess": Constant(1e-4),
    "algorithm": "Picard",
}

mesh = Mesh("../meshes/ITER.msh", dim = 2, distribution_parameters={"partition": False}, reorder = True)
params["mesh"] = mesh

In [20]:
from solver import GradShafranovSolver
solver = GradShafranovSolver(params)


Loading mesh for geometry 'ITER'...


Error: error code 65
[0] DMPlexCreateFromFile() at /opt/petsc/src/dm/impls/plex/plexcreate.c:6249
[0] DMPlexCreateGmshFromFile() at /opt/petsc/src/dm/impls/plex/plexgmsh.c:1511
[0] PetscViewerFileSetName() at /opt/petsc/src/sys/classes/viewer/impls/ascii/filev.c:764
[0] PetscViewerFileSetName_ASCII() at /opt/petsc/src/sys/classes/viewer/impls/ascii/filev.c:868
[0] Unable to open file
[0] Cannot open PetscViewer file: ./meshes/ITER.msh due to "No such file or directory"